In [1]:
# In this notebook we test how our method works on the Burgers equation. Mostly meant as an exploration w.r.t noise, 
# dataset size etc.

In [12]:
# General imports
import numpy as np
import torch

# DeepMoD stuff
from deepymod_torch import DeepMoD
from deepymod_torch.model.func_approx import NN, Siren
from deepymod_torch.model.library import Library1D
from deepymod_torch.model.constraint import LeastSquares
from deepymod_torch.model.sparse_estimators import Threshold
from deepymod_torch.training import train_split_full
from deepymod_torch.training.sparsity_scheduler import TrainTestPeriodic

from phimal_utilities.data import Dataset
from phimal_utilities.data.burgers import BurgersDelta

if torch.cuda.is_available():
    device ='cuda'
else:
    device = 'cpu'
    
# Settings for reproducibility
np.random.seed(42)
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
v = 0.1
A = 1.0

x = np.linspace(-3, 4, 100)
t = np.linspace(0.5, 5.0, 50)
x_grid, t_grid = np.meshgrid(x, t, indexing='ij')
dataset = Dataset(BurgersDelta, v=v, A=A)
    
X, y = dataset.create_dataset(x_grid.reshape(-1, 1), t_grid.reshape(-1, 1), n_samples=1000, noise=0.6, random=True, normalize=False)
X, y = X.to(device), y.to(device)

In [7]:
network = NN(2, [30, 30, 30, 30, 30], 1)
library = Library1D(poly_order=2, diff_order=3) # Library function
estimator = Threshold(0.1) #Clustering() # Sparse estimator 
constraint = LeastSquares() # How to constrain
model = DeepMoD(network, library, estimator, constraint).to(device) # Putting it all in the model
  
sparsity_scheduler = TrainTestPeriodic(patience=200, periodicity=50)
optimizer = torch.optim.Adam(model.parameters(), betas=(0.99, 0.999), amsgrad=True, lr=2e-3) # Defining optimizer

In [9]:
train_split_full(model, X, y, optimizer, sparsity_scheduler, log_dir='data/1000_0.6/', write_iterations=25, max_iterations=25000, delta=0.001, patience=200) # Runni

| Iteration | Progress | Time remaining |     Loss |      MSE |      Reg |    L1 norm |
       6050     24.20%             882s   1.22e-02   1.21e-02   1.34e-04   3.97e+00 

KeyboardInterrupt: 

not bad, but it diverges at the end; maybe due to too high lr?

In [10]:
network = NN(2, [30, 30, 30, 30, 30], 1)
library = Library1D(poly_order=2, diff_order=3) # Library function
estimator = Threshold(0.2) #Clustering() # Sparse estimator 
constraint = LeastSquares() # How to constrain
model = DeepMoD(network, library, estimator, constraint).to(device) # Putting it all in the model
  
sparsity_scheduler = TrainTestPeriodic(patience=200, periodicity=50)
optimizer = torch.optim.Adam(model.parameters(), betas=(0.99, 0.999), amsgrad=True, lr=2e-3) # Defining optimizer

In [11]:
train_split_full(model, X, y, optimizer, sparsity_scheduler, log_dir='data/1000_0.6_threshold_0.2/', write_iterations=25, max_iterations=25000, delta=0.001, patience=200) # Runni

| Iteration | Progress | Time remaining |     Loss |      MSE |      Reg |    L1 norm |
       6000     24.00%             916s   1.24e-02   1.22e-02   1.24e-04   1.56e+00 Algorithm converged. Stopping training.


Nice, it works. Now let's try with SIREN?

In [24]:
v = 0.1
A = 1.0

x = np.linspace(-3, 4, 100)
t = np.linspace(0.5, 5.0, 50)
x_grid, t_grid = np.meshgrid(x, t, indexing='ij')
dataset = Dataset(BurgersDelta, v=v, A=A)
    
X, y = dataset.create_dataset(x_grid.reshape(-1, 1), t_grid.reshape(-1, 1), n_samples=1000, noise=0.6, random=True, normalize=True)
X, y = X.to(device), y.to(device)

In [25]:
network = Siren(2, [30, 30, 30, 30, 30], 1)
library = Library1D(poly_order=2, diff_order=3) # Library function
estimator = Threshold(0.2) #Clustering() # Sparse estimator 
constraint = LeastSquares() # How to constrain
model = DeepMoD(network, library, estimator, constraint).to(device) # Putting it all in the model
  
sparsity_scheduler = TrainTestPeriodic(patience=200, periodicity=50)
optimizer = torch.optim.Adam(model.parameters(), betas=(0.99, 0.999), amsgrad=True, lr=2e-3) # Defining optimizer

In [ ]:
train_split_full(model, X, y, optimizer, sparsity_scheduler, test='mse', log_dir='data/1000_0.6_threshold_0.2_siren/', write_iterations=25, max_iterations=25000, delta=0.001, patience=200) # Runni

That doesnt work at all, let's put the threshold lower?

In [29]:
network = Siren(2, [30, 30, 30, 30, 30], 1)
library = Library1D(poly_order=2, diff_order=3) # Library function
estimator = Threshold(0.1) #Clustering() # Sparse estimator 
constraint = LeastSquares() # How to constrain
model = DeepMoD(network, library, estimator, constraint).to(device) # Putting it all in the model
  
sparsity_scheduler = TrainTestPeriodic(patience=1000, periodicity=50)
optimizer = torch.optim.Adam(model.parameters(), betas=(0.9, 0.999), amsgrad=True, lr=5e-4) # Defining optimizer

In [30]:
train_split_full(model, X, y, optimizer, sparsity_scheduler, test='mse',log_dir='data/1000_0.6_threshold_0.1_siren_2/', write_iterations=25, max_iterations=25000, delta=0.001, patience=200) # Runni

| Iteration | Progress | Time remaining |     Loss |      MSE |      Reg |    L1 norm |
       4125     16.50%            1037s   5.21e-03   4.71e-03   4.95e-04   2.16e+00 

KeyboardInterrupt: 

Nope....